In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
ratings = pd.DataFrame([[1,2,8,9,3,3],[2,1,9,8,4,2],[2,2,6,8,2,3],[9,7,2,3,1,1],
                   [1,1,1,2,8,7],[2,2,3,2,8,8],[7,9,2,2,2,3],[9,8,2,3,1,3]])

users = np.array(["u0","u1","u2","u3","u4","u5","u6","u7"])
movies = np.array(["horror1","horror2","drama1","drama2","art1","art2"])

ratings.columns = movies
ratings.index = users

In [3]:
ratings

,horror1,horror2,drama1,drama2,art1,art2
u0,1,2,8,9,3,3
u1,2,1,9,8,4,2
u2,2,2,6,8,2,3
u3,9,7,2,3,1,1
u4,1,1,1,2,8,7
u5,2,2,3,2,8,8
u6,7,9,2,2,2,3
u7,9,8,2,3,1,3


It is possible some users are more critical in rating movies than others and so we normalize/center the data.

In [4]:
mean_ratings = ratings.mean(axis=1).to_dict()
mean_ratings

{'u0': 4.333333333333333,
 'u1': 4.333333333333333,
 'u2': 3.8333333333333335,
 'u3': 3.8333333333333335,
 'u4': 3.3333333333333335,
 'u5': 4.166666666666667,
 'u6': 4.166666666666667,
 'u7': 4.333333333333333}

In [5]:
df = pd.DataFrame(ratings.values - ratings.mean(axis=1).values.reshape(-1,1)).round(2)

df.columns = ["horror1","horror2","drama1","drama2","art1","art2"]
df.index = ["u0","u1","u2","u3","u4","u5","u6","u7"]

df

,horror1,horror2,drama1,drama2,art1,art2
u0,-3.33,-2.33,3.67,4.67,-1.33,-1.33
u1,-2.33,-3.33,4.67,3.67,-0.33,-2.33
u2,-1.83,-1.83,2.17,4.17,-1.83,-0.83
u3,5.17,3.17,-1.83,-0.83,-2.83,-2.83
u4,-2.33,-2.33,-2.33,-1.33,4.67,3.67
u5,-2.17,-2.17,-1.17,-2.17,3.83,3.83
u6,2.83,4.83,-2.17,-2.17,-2.17,-1.17
u7,4.67,3.67,-2.33,-1.33,-3.33,-1.33


In [6]:
#making predictions for (userid, movieid) pair
userid = 'u5'
movieid = 'drama1' 

df1 = df.copy()
df1.loc[userid, movieid]=None
df1

,horror1,horror2,drama1,drama2,art1,art2
u0,-3.33,-2.33,3.67,4.67,-1.33,-1.33
u1,-2.33,-3.33,4.67,3.67,-0.33,-2.33
u2,-1.83,-1.83,2.17,4.17,-1.83,-0.83
u3,5.17,3.17,-1.83,-0.83,-2.83,-2.83
u4,-2.33,-2.33,-2.33,-1.33,4.67,3.67
u5,-2.17,-2.17,NaN,-2.17,3.83,3.83
u6,2.83,4.83,-2.17,-2.17,-2.17,-1.17
u7,4.67,3.67,-2.33,-1.33,-3.33,-1.33


In [7]:
df1 = df1.dropna(axis='columns')
df1

,horror1,horror2,drama2,art1,art2
u0,-3.33,-2.33,4.67,-1.33,-1.33
u1,-2.33,-3.33,3.67,-0.33,-2.33
u2,-1.83,-1.83,4.17,-1.83,-0.83
u3,5.17,3.17,-0.83,-2.83,-2.83
u4,-2.33,-2.33,-1.33,4.67,3.67
u5,-2.17,-2.17,-2.17,3.83,3.83
u6,2.83,4.83,-2.17,-2.17,-1.17
u7,4.67,3.67,-1.33,-3.33,-1.33


In [8]:
def find_cosine_similarity(v1,v2):
    return np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [9]:
def find_cosine_similarity_dict(userid, df1):

    v1 = df1.loc[userid].values
    csim = {}

    for user in df1.index:

        if (user!=userid):
            v2 = df1.loc[user].values
            csim[user] = find_cosine_similarity(v1,v2)
            
    return csim

In [10]:
csim = find_cosine_similarity_dict(userid, df1)
csim

{'u0': -0.18846848573643268,
 'u1': -0.14937219947060032,
 'u2': -0.3230407133410321,
 'u3': -0.7876091452135945,
 'u4': 0.9848779179101157,
 'u6': -0.5773762616126623,
 'u7': -0.7098377347000132}

In [11]:
def make_prediction(df, userid, movieid, csim):

    num=0
    den=0

    for user in df.index:

        if (user!=userid):
            r = df.loc[user,movieid]
            c_similarity = csim[user]

            num += r * c_similarity
            den += c_similarity

    prediction = mean_ratings[userid] + num/den
    return prediction

In [12]:
prediction = make_prediction(df, userid, movieid, csim)
prediction

4.187718588714821

In [13]:
print('ACTUAL RATING: ', ratings.loc[userid, movieid])
print('PREDICTED RATING: ', prediction.round(2))

ACTUAL RATING:  3
PREDICTED RATING:  4.19


**Let's calculate accuracy of this method by running the method for a large number of examples**

In [14]:
def iteration(userid, movieid, df):
    
    df1 = df.copy()
    df1.loc[userid, movieid]=None
    df1 = df1.dropna(axis='columns')

    csim = find_cosine_similarity_dict(userid, df1)

    prediction = make_prediction(df, userid, movieid, csim)
    actual = ratings.loc[userid, movieid]
    error = abs(actual-prediction)
    
    return error

In [15]:
N_ITERATIONS = 1000

errors=[]

for _ in tqdm(range(N_ITERATIONS)):
    u = np.random.choice(users)
    m = np.random.choice(movies)

    error = iteration(u, m, df)
    errors.append(error)
    
errors = np.array(errors)
mae = np.mean(errors).round(2)
rmse = np.sqrt(np.mean(errors**2)).round(2)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 275.38it/s]


In [16]:
print('Mean Absolute Error (MAE): ', mae)
print('Root Mean Square Error (RMSE): ', rmse)

Mean Absolute Error (MAE):  34.98
Root Mean Square Error (RMSE):  82.42


**Similarly can be done for item-item collaborative filtering**